In [1]:
#preprocessing

import json
import pandas as pd
import random

In [2]:
# JSON 파일에서 데이터를 불러오는 함수
def load(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return None

In [3]:
def count_ratings(data):
    user_counts = {}
    item_counts = {}
    for entry in data:
        user_id = entry['user_id']
        item_id = entry['business_id']
        user_counts[user_id] = user_counts.get(user_id, 0) + 1
        item_counts[item_id] = item_counts.get(item_id, 0) + 1
    return user_counts, item_counts

In [4]:
def filter_users_with_min_ratings(data, min_ratings=2):
    # 사용자별 평가 수 계산
    user_counts = {}
    for entry in data['train'] + data['test']:
        user_id = entry['user_id']
        user_counts[user_id] = user_counts.get(user_id, 0) + 1

    # 평가 수가 min_ratings 이상인 사용자 필터링
    active_users = {user for user, count in user_counts.items() if count >= min_ratings}

    # 필터링된 사용자를 기준으로 'train' 및 'test' 데이터 필터링
    filtered_train_data = [item for item in data['train'] if item['user_id'] in active_users]
    filtered_test_data = [item for item in data['test'] if item['user_id'] in active_users]

    # 필터링된 데이터 업데이트
    data['train'] = filtered_train_data
    data['test'] = filtered_test_data

    return data

In [5]:
# 데이터 추출 함수
def extract(data, max_items=50000):
    if isinstance(data, dict):
        # 전체 데이터에서 사용자별, 아이템별 평가 수 계산
        user_counts, item_counts = count_ratings(data.get('train', []))

        # 평가가 2개 이상인 사용자만 필터링
        active_users = {user for user, count in user_counts.items() if count > 2}

        # 상위 N명의 사용자와 상위 M개의 아이템 필터링
        top_users = set(sorted(active_users, key=user_counts.get, reverse=True)[:1000])
        top_items = set(sorted(item_counts, key=item_counts.get, reverse=True)[:500])

        # 'train' 데이터 필터링
        filtered_train_data = [item for item in data['train'] if item['user_id'] in top_users and item['business_id'] in top_items]

        subset_data = {}
        # 필터링된 'train' 데이터 랜덤 추출
        if len(filtered_train_data) > max_items:
            subset_data['train'] = random.sample(filtered_train_data, max_items)
        else:
            subset_data['train'] = filtered_train_data

        # 추출된 'train' 데이터의 'business_id'를 기준으로 'test' 데이터 필터링
        train_business_ids = set(item['business_id'] for item in subset_data['train'])
        if 'test' in data:
            subset_data['test'] = [item for item in data['test'] if item['business_id'] in train_business_ids]
        
        filtered_subset_data = filter_users_with_min_ratings(subset_data)
        
        return filtered_subset_data
    else:
        print("The data is not in the expected dictionary format.")
        return None


In [6]:
# 파일 경로
file_path = r'./data/filter_all_t.json'  # 원본 파일 경로
output_file_path = r'./data/filter_subset.json'  # 결과 파일 경로

# 파일에서 JSON 데이터 불러오기
loaded_json_data = load(file_path)

In [7]:
# 데이터가 성공적으로 불러와졌는지 확인 후 랜덤 부분 추출 실행
if loaded_json_data:
    subset_data = extract(loaded_json_data)
    
    # 추출된 부분을 새로운 파일로 저장
    if subset_data:
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            json.dump(subset_data, output_file)
        print(f"Subset extracted and saved to {output_file_path}")
else:
    print("No data loaded")

Subset extracted and saved to ./data/filter_subset.json


In [8]:

# 추출된 데이터 로드 및 처리
try:
    with open(output_file_path, 'r', encoding='utf-8') as file:
        processed_data = json.load(file)
        
    # 'train'과 'test' 데이터를 분리하여 데이터프레임 생성
    df_train = pd.DataFrame(processed_data['train'])
    df_test = pd.DataFrame(processed_data['test'])

    # 피벗 테이블 생성
    pivot_train = df_train.pivot(index='user_id', columns='business_id', values='rating').fillna(0)
    pivot_test = df_test.pivot(index='user_id', columns='business_id', values='rating').fillna(0)

    print(pivot_train)
    print(pivot_test)

except Exception as e:
    print(f"An error occurred while processing the subset file: {e}")


business_id            6040a833c6fcf1fddba0db65  6040c0517cd8bf130361fe41  \
user_id                                                                     
100003947933664632255                       0.0                       0.0   
100034227509716251848                       0.0                       0.0   
100075841280804377681                       0.0                       0.0   
100124627392132660033                       0.0                       0.0   
100186511506901662710                       0.0                       0.0   
...                                         ...                       ...   
118255687420082471679                       0.0                       0.0   
118306453335411440689                       0.0                       0.0   
118422090695402195892                       0.0                       0.0   
118433659701788556614                       0.0                       0.0   
118444412062604295521                       0.0                       0.0   